# Retrieve underlying RICs of chain RIC

## Method 2) Get chain constituents with Refinitiv Data Platform API
Here comes an easier and simpler way! with only one endpoint in RDP API, we can retrieve constituent lists of Chain RIC in one call (not counting a call to retrieve access token though). The example of this API call for this endpoint can be found in the API Playground: /data/pricing/chains/v1/

With this method, we can retrieve the chain constituents list with just one API call instead of looping through each Chain Record and sending requests to the server multiple times. We also don't need to use WebSocket to make a connection to the Real-time platform so no need to implement the message handler functions. Lastly, we don't have to know the chain structure and handle each chain's record template because this endpoint is already handling it for us!

In [3]:
import rdpToken
import http.client
import json
import time

def get_underlying_rics_rdp(base_ric):
    try:
        auth_token = rdpToken.getToken()
        
        print(f'Retrieving underlying RICs of chain RIC: ', base_ric)
        conn = http.client.HTTPSConnection("api.refinitiv.com")
        payload = ''
        headers = {
            'Authorization': 'Bearer ' + auth_token
        }
        conn.request('GET', f'/data/pricing/chains/v1/?universe={base_ric}&limit=100', payload, headers)
        r = conn.getresponse()
        data = r.read()

        data = json.loads(data.decode("utf-8"))

    except requests.exceptions.RequestException as e:
        print('Refinitiv Data Platform authentication exception failure:', e)

    if r.status == 200:
        print('Underlying RICs of chain are successfully retrieved')

    elif r.status in [400, 401, 403, 404, 410, 451, 503]:
        print('Error retrieving underlying RICs (to be retried):', r.status, r.reason)
        time.sleep(5)
        return get_underlying_rics_rdp()

    elif data == b'':
        print('Error retrieving underlying RICs (to be retried): get empty response')
        time.sleep(5)
        return get_underlying_rics_rdp()

    else:
        # Retry the request to the API gateway
        print('Retrieval of underlying RICs of chain HTTP code:', r.status, r.reason)
        time.sleep(5)
        print('Retry change request')
        return get_underlying_rics_rdp()
    return data

In [4]:
if __name__ == "__main__":
    base_ric = '0#.FTSE'
    data = get_underlying_rics_rdp(base_ric)
    print(str(len(data['data']['constituents'])) + ' underlying RICs are retrieved, which are')
    print(data['data']['constituents'])

Existing token read from: token.txt
Retrieving underlying RICs of chain RIC:  0#.FTSE
Underlying RICs of chain are successfully retrieved
102 underlying RICs are retrieved, which are
['ABF.L', 'BAES.L', 'AZN.L', 'BARC.L', 'AAL.L', 'AV.L', '.FTSE', 'AAF.L', '.AD.FTSE', 'AHT.L', 'AUTOA.L', 'ANTO.L', 'ABDN.L', 'ADML.L', 'BNZL.L', 'BMEB.L', 'CCH.L', 'CNA.L', 'BRBY.L', 'BP.L', 'CRDA.L', 'BLND.L', 'BEZG.L', 'CPG.L', 'BT.L', 'BATS.L', 'BKGH.L', 'BDEV.L', 'FRES.L', 'GSK.L', 'GLEN.L', 'FRAS.L', 'DGE.L', 'FCIT.L', 'FLTRF.L', 'CRH.L', 'EXPN.L', 'HLMA.L', 'CTEC.L', 'EDV.L', 'ENT.L', 'DCC.L', 'HSBA.L', 'ICAG.L', 'IHG.L', 'KGF.L', 'IMB.L', 'ITRK.L', 'HLN.L', 'HSX.L', 'III.L', 'JMAT.L', 'JD.L', 'LAND.L', 'HRGV.L', 'INF.L', 'LLOY.L', 'LGEN.L', 'NWG.L', 'MNG.L', 'PRU.L', 'NG.L', 'PSHP.L', 'PHNX.L', 'PSN.L', 'MNDI.L', 'OCDO.L', 'MRON.L', 'LSEG.L', 'NXT.L', 'REL.L', 'SJP.L', 'SHEL.L', 'RS1R.L', 'RR.L', 'RIO.L', 'PSON.L', 'SGE.L', 'RKT.L', 'RMV.L', 'SDR.L', 'SBRY.L', 'SGRO.L', 'RTO.L', 'SPX.L', 'SSE.L', '

## Thank you